# How to generate independent feature sets using Network Graphs

### Introduction

One of the core assumptions underpinning many statistical models is independence among the explanatory variables. So let's assume that you want to train your models with variables that are no more than .1 correlated with any other variable in the set. Sounds easy, no?

Not quite. You could manually select your variable combinations but that gets taxing quite quickly and only works when you have a low variable count. But let's say you have a modest count of 8 variables. If you choose 2 variables from 8, that's 56 unique pairings. That's simple enough: just look at a corrlation matrix and choose only the pairs below our .1 threshold.

But what about groups of 3? That's 336 unique combinations (8 chooses 3 i.e. 8!/(3!(8-3)!)). Furthermore, correlation matrices are pairwise so not only do you need to find the pairs that are uncorrelated, but you need to make sure that no variable is more than .1 correlated with any of the other variables. For instance, if you have variables 1, 2, and 3 you'd need to manually check that (1,2), (1,3), and (2,3) all have correlations below .1

And that's just for a feature set of 3! Doing this for 4 is way more taxing (1680 combos), 5 (6720 combos), 6 (2160 combos) and so on.

Most data scientists are working with feature sets much larger than the 8 I describe above. 

##### How can we train multiple models on feature sets of varying size and ensure that none of the feature sets violate our independence assumption?

### Variance Inflation Factor

A common way to check for the occurence of multicollinearity in a regression model (variable count > 2) is by calculating what's called the Variance Inflation Factor. VIF "is calculated by taking the the ratio of the variance of all a given model's betas divide by the variance of a single beta if it were fit alone." [reference_here](https://etav.github.io/python/vif_factor_python.html). Simply put: VIF attempts to capture how well the rest of a model's variables explain the independent variable you isolated.

Variance Inflation Factor calculations are important because they capture multi-collinearity even among variable sets that see low pairwise correlations. So in no way am I suggesting that the method I discuss below is superior/intended to replace it.

### Filtering Large Feature Sets

But let's assume we have 100 features and we want to experiment with different stacking strategies. In one strategy, we pool a large number of weakly trained ensemble/logit models trained on only a fraction of the feature space. In another strategy, we train a few models on a larger feature set. For both strategies we use the base model outputs as inputs to our higher level model. And also assume that we want to test the efficacy of a one-model approach.

If we have 100 features we are left with a very large number of feature combinations that we'll want our script to run through. Furthermore, running VIF analysis on dozens of variables for hundreds if not thousands of models is pretty costly. So how can we filter down the feature space to something more manageable?

### Defining the Challenge

I suggested at the beginning that we consider each feature set of varying sizes where all variables contained within the set are no more correlated with any other variable within the set than a stipulated threshold. For simplicity let's just say that the threshold is a correlation of .1

Each set then will be comprised of these variables. So let's assume that we have three uncorrelated features: A, B, C. This means:

- (A,B), (A,C) and (B,C) all have a correlation < .1

So in this case we can train our model on these three pairwise combinations and we can also train on all three variables.

Similarly, we see that a set of size 4 can be decomposed into sets of size 3 and 2. Consider A, B, C, D:

- (A, B, C), (A, B, D), (A, C, D), (B, C, D) acceptable triangles
- (A,B), (A,C), (A,D), (B,C), (B,D), (C,D) acceptable pairwise

And so on.

We need some way to identify all acceptable groups of varying size. For instance, when testing our strategy of training and stacking many low-dimensional models, we'll need feature sets of size 1-6. However, for training higher dimensional models we'll need to identify feature sets much larger (if they exist).

### Network Graphs

I'll close this exposition by discussing how I go about doing this and then I'll provide a walkthrough of my code. Our challenge is three-fold:

1. We need to identify all lower-level feature set combinations (the decompositions of a larger set)
2. We need to traverse a very large feature space
3. We need to do this quickly

My theory is that you can treat each set of uncorrelated variables as a complete network, meaning that each node is connected to every other node in the subnetwork. My strategy then is as follows:

- Identify all pairs in a correlation matrix that do no exceed our threshold
- Convert these pairwise points into a network dictionary
- Mine the network for all instances of complete subnetworks of any size.

### The Code

I'm going to start by loading two data sets: one containing 25 features, the other containing 127. Both are engineered versions of the Kaggle titanic dataset and have already been normalized.

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
small_df = pd.read_csv("data/train_norm.csv", sep=",")
large_df = pd.read_csv("data/train_norm_large.csv", sep=",")

In [3]:
# remove "Survived"
small_df = small_df.drop("Survived", axis=1)
large_df = large_df.drop("Survived", axis=1)

In [4]:
print("Number of features in small feature set: {}".format(len(small_df.columns)))
print("Number of features in large feature set: {}".format(len(large_df.columns)))

Number of features in small feature set: 25
Number of features in large feature set: 127


##### Plotting our correlations

Let's get a view of our correlation matrices

In [5]:
# convert df to np.array
small = small_df.as_matrix()
large = large_df.as_matrix()

# print small correlation matrix
pd.DataFrame(np.corrcoef(small.T))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.000000,-0.094706,-0.103837,-0.246762,0.124148,0.095006,-0.210181,0.007803,0.067836,-0.079175,...,0.134012,-0.268843,0.116998,0.014372,0.081493,0.126311,0.037751,0.069139,-0.335746,0.036142
1,-0.094706,1.000000,-0.091723,-0.101397,0.031540,0.047170,-0.070402,0.057869,0.000701,-0.057067,...,0.169757,-0.100738,-0.081106,0.000815,0.022539,0.034859,0.016682,0.028670,-0.076085,-0.035789
2,-0.103837,-0.091723,1.000000,-0.123364,0.097580,0.084204,-0.176511,0.086845,-0.040976,-0.077763,...,-0.000541,0.049242,0.118489,0.002435,-0.107925,0.105966,0.034776,0.047261,-0.039550,-0.155727
3,-0.246762,-0.101397,-0.123364,1.000000,-0.143544,-0.112363,0.108507,-0.136621,-0.136907,0.088725,...,-0.707618,0.857738,-0.581027,-0.070150,-0.046340,-0.129952,-0.055360,-0.109063,0.049980,-0.020764
4,0.124148,0.031540,0.097580,-0.143544,1.000000,-0.288145,-0.621220,0.307655,-0.149344,-0.171537,...,0.041842,-0.137774,0.110023,0.176303,-0.241679,0.459163,0.524565,-0.185458,-0.356497,-0.126442
5,0.095006,0.047170,0.084204,-0.112363,-0.288145,1.000000,-0.560901,-0.124442,-0.122618,0.190507,...,0.007017,-0.131846,0.142598,0.054700,-0.074078,0.124750,-0.153057,0.639519,-0.321464,-0.115498
6,-0.210181,-0.070402,-0.176511,0.108507,-0.621220,-0.560901,1.000000,-0.152124,0.243459,-0.007930,...,0.037656,0.130900,-0.145158,-0.187849,0.273336,-0.493703,-0.324302,-0.352973,0.587471,0.211628
7,0.007803,0.057869,0.086845,-0.136621,0.307655,-0.124442,-0.152124,1.000000,-0.138898,-0.781418,...,0.064391,-0.119726,0.094321,0.071444,0.288894,0.174953,-0.004069,-0.135887,-0.257858,-0.102986
8,0.067836,0.000701,-0.040976,-0.136907,-0.149344,-0.122618,0.243459,-0.138898,1.000000,-0.492727,...,0.213648,-0.134138,-0.045748,0.002381,-0.137222,-0.061058,-0.085933,-0.054312,0.327503,-0.059930
9,-0.079175,-0.057067,-0.077763,0.088725,-0.171537,0.190507,-0.007930,-0.781418,-0.492727,1.000000,...,-0.103434,0.084951,0.018451,-0.055445,-0.161972,-0.113074,0.060780,0.161509,0.042654,0.138988


And of course our larger feature set will be much bigger!

##### Let's now identify all pairs that fall below our .1 threshold

In [6]:
def uncorrPair(matrix, threshold):
	# return grouped indices
	# for nxn matrix
	result = []
	m_size = len(matrix)
	for i in range(m_size):
		for j in range(m_size):
			if matrix[i][j] < threshold:
				if sorted((j,i)) not in result:
					result.append(list(sorted((i,j))))
	return list(result)

In [7]:
# generate list of uncorrelated pairs
small_pairs = uncorrPair(np.absolute(np.corrcoef(small.T)), .1)
large_pairs = uncorrPair(np.absolute(np.corrcoef(large.T)), .1)
print(small_pairs[:10])
print(large_pairs[:10])

[[0, 1], [0, 5], [0, 7], [0, 8], [0, 9], [0, 10], [0, 13], [0, 14], [0, 18], [0, 19]]
[[0, 4], [0, 13], [0, 14], [0, 15], [0, 17], [0, 19], [0, 20], [0, 21], [0, 22], [0, 24]]


##### This is where the fun begins. Let's define a class Network which will mine these pairs for complete networks

In [8]:

class Network:
	def __init__(self):
		"""
		Initiate class attributes
		:attr .network: holds a dictionary of node connections
		:attr ._pairs: currently this class ingests a list of list pairs
			[[0,1], [0,2], ..., [x,y]]
		:attr .complete_nodes: list to hold list of nodes that form complete network
			Currently, the only purpose of this class is to:
				1. generate node mapping
				2. mine the network graph for all complete subnetworks of any size
					My networks are bidirectional and assume no weights
		"""
		self.network = {}
		self._pairs = None
		self.complete_subnet = []

	def buildNetwork(self, data):
		"""
		Currently this method takes in list of lists (sublists could be of > 1d)
		:param data: list of list holding pair data (could be more)
		"""

		# store list of list as ._pairs so we can reference in completenessSearch()
		self._pairs = data

		# store list of unique nodes
		nodes = list(set(item for pair in data for item in pair))

		# iterate through coordinate data and store node connections in list
		# associated with each dictionary node key
		for n in nodes:
			con = []
			for d in data:
				if n in d:
					for i in d:
						if i != n:
							con.append(i)
							break
			# sort the list and store
			self.network[n] = list(sorted(con))

	def completenessSearch(self):
		"""
		The method recursively searches for all
		complete networks present in self.network
		"""
		def treeSearch(network, shared_match, memory):
			if shared_match == []:
				return memory
			else:
				memory.append(memory[-1] + [shared_match[0]])
				set_list = [set(network[i]) for i in memory[-1]]
				shared_match = list(set.intersection(*set_list))
				return treeSearch(network, shared_match, memory)

		for p in self._pairs:
			start_match, start_mem = [p[0]], [[p[1]]]
			self.complete_subnet += treeSearch(self.network, start_match, start_mem)
		# remove duplicates
		self.complete_subnet = [list(g) for g in set(tuple(g) for g in self.complete_subnet)]

And let's convert our pairwise data to a network mapping:

In [9]:
# initiate our networks
small_net, large_net = Network(), Network()

# map pairwise data
small_net.buildNetwork(small_pairs)
large_net.buildNetwork(large_pairs)

print(small_net.network)

{0: [1, 5, 7, 8, 9, 10, 13, 14, 18, 19, 21, 22, 24], 1: [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24], 2: [1, 4, 5, 7, 8, 9, 10, 11, 13, 15, 16, 18, 21, 22, 23], 3: [9, 10, 11, 12, 14, 18, 19, 21, 23, 24], 4: [1, 2, 10, 11, 13, 14, 15], 5: [0, 1, 2, 11, 12, 13, 14, 15, 18, 19], 6: [1, 9, 10, 12, 13, 14, 15], 7: [0, 1, 2, 10, 13, 14, 15, 17, 18, 21], 8: [0, 1, 2, 10, 13, 14, 17, 18, 20, 21, 22, 24], 9: [0, 1, 2, 3, 6, 11, 12, 13, 14, 16, 17, 18, 21, 23], 10: [0, 1, 2, 3, 4, 6, 7, 8, 12, 13, 14, 15, 18, 19, 21, 22, 23, 24], 11: [1, 2, 3, 4, 5, 9, 13, 14, 15, 16, 17, 18], 12: [1, 3, 5, 6, 9, 10, 13, 14, 15, 16, 17, 18], 13: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 18, 20, 21, 22, 23, 24], 14: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 24], 15: [2, 4, 5, 6, 7, 10, 11, 12, 14, 18, 19, 20, 21, 22, 23, 24], 16: [2, 9, 11, 12, 19, 21, 23, 24], 17: [1, 7, 8, 9, 11, 12, 14, 18, 19, 20, 21, 23, 24], 18: [0, 1, 2, 3, 5, 7, 8, 9, 10

Above is the network mapping for our small features. Each key is a node and each list are its pairwise connections.

##### Mining complete subnetworks

I built a method, completenessSearch() that uses a local recursive function to identify each complete subnetwork in a larger complete subnetwork. Once it cannot expand beyond a subnetwork recursion ends and moves on to the next beginner node. Because we're mining complete networks, all I have to do is keep running set intersections to find the next subnetwork level. 

You will see that the subnetwork mining has a very low runtime despite the fact that it is not optimized.

In [10]:
# run small search
start_time = time.time()
small_net.completenessSearch()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.012594223022460938 seconds ---


In [11]:
# and let's print some of our uncorrelated feature sets
print(small_net.complete_subnet[:50])
print("Number of uncorrelated feature sets: {}".format(len(small_net.complete_subnet)))

[[10, 1, 0, 7, 18, 13, 21, 14], [16, 9], [14, 6, 1, 9, 12, 13], [19, 10, 0, 1], [18, 8, 0, 1, 10, 13, 24, 21, 14], [21, 8, 0, 1, 10, 13, 24, 18, 14], [12, 5, 1], [13, 10, 0, 1, 7], [5, 2, 1], [18, 3, 9], [23, 15, 18, 2], [9, 1, 0, 18, 13], [24], [22, 14, 0, 1, 8, 10, 24, 18, 21], [14, 8, 0, 1, 10, 13, 24, 18, 21, 22], [10, 1, 0, 7, 18, 13, 21], [9, 0], [13, 9, 0, 1, 18, 21, 14], [24, 17, 1, 8, 18, 21], [7, 0, 1, 10], [22, 2, 1, 8, 21, 10], [10, 7], [18, 8, 0, 1, 10, 13], [19, 10, 0], [24, 14], [14, 1], [9, 0, 1, 18, 13, 21], [22, 14, 0, 1, 8, 10], [15, 4], [11, 5, 1], [21, 7, 0, 1, 10, 18], [23, 9, 1, 2, 18, 13], [13, 8, 0, 1, 10, 14, 24, 18, 21], [22, 10, 0], [12, 5, 1, 18, 13], [18, 12, 1, 5, 13], [14, 5, 0, 1], [23, 1, 2, 9], [21, 1, 0, 7, 10, 18, 13, 14], [13, 12, 1, 5, 18], [24, 3, 10, 18, 21, 14], [17, 8, 1, 14, 24, 18], [14, 13, 0, 1, 5, 18], [14, 10, 0], [24, 22, 0, 1, 8, 10, 18, 13], [7, 2, 1, 21, 10, 18, 13], [22, 21, 0, 1, 8, 10, 24, 18, 13], [20, 14], [18, 10], [18, 17, 1, 

In [12]:
# let's see the runtime for the large feature set
start_time = time.time()
large_net.completenessSearch()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.11841607093811035 seconds ---


In [13]:
print(large_net.complete_subnet[:50])
print("Number of uncorrelated feature sets: {}".format(len(large_net.complete_subnet)))

[[87, 86, 0, 98, 101, 108], [74, 14, 0, 98, 4], [98, 12, 96, 101, 108, 86], [104, 2, 96, 33, 103, 91, 109], [62, 31], [123, 31], [34, 16, 98], [109, 32], [113, 105, 34, 27, 15], [99, 22], [120, 33, 96, 91, 27, 22], [14, 7, 91], [109, 102, 0, 96, 97], [108, 94, 0, 96], [47, 26, 0], [101], [110, 15, 96], [81, 19, 0, 26, 109, 22], [14, 3, 86], [87, 14, 0, 98, 108, 91], [34, 27, 98], [91, 28, 2], [120, 31, 0, 96, 98, 27], [52, 22, 0, 91], [24, 15, 0], [103, 27], [22, 7], [113, 22, 0, 101], [35, 15, 0, 96], [59, 14], [98, 93, 108, 14], [89, 15, 105], [28, 22, 2], [85, 29, 98], [42, 15, 0], [97, 96, 0, 98], [44, 15], [27, 22, 0, 96, 35, 91, 95], [105, 15], [91, 44], [27, 24, 0, 96, 98], [111, 33], [81, 22], [126, 91, 0], [110, 98, 96, 97, 103, 108, 109, 111, 91, 95], [26, 4, 0, 101, 103, 108], [70, 22, 0, 26, 19], [77, 34], [63, 19], [58, 22, 0, 14]]
Number of uncorrelated feature sets: 5852


In [14]:
# randomly generate 10000, 100000, 1000000 pairs and run network
from random import randint
ten_pair = [[randint(0,1000), randint(0,1000)] for i in range(0,10000)]
hun_pair = [[randint(0,10000), randint(0,10000)] for i in range(0,100000)]

In [17]:
ten_net = Network()
ten_net.buildNetwork(ten_pair)

In [18]:
# time how long it takes for 10,000
start_time = time.time()
ten_net.completenessSearch()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.21075797080993652 seconds ---


In [19]:
hun_net = Network()
hun_net.buildNetwork(hun_pair)

In [20]:
# time how long it takes for 100,000
start_time = time.time()
hun_net.completenessSearch()
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.789815902709961 seconds ---


In [21]:
# one million
mil_pair = [[randint(0,100000), randint(0,100000)] for i in range(0,1000000)]

In [22]:
mil_net = Network()
mil_net.buildNetwork(mil_pair)

In [23]:
start_time = time.time()
mil_net.completenessSearch()
print("--- %s seconds ---" % (time.time() - start_time))

--- 20.978532075881958 seconds ---


### Conclusion

The network submining approach I run above is a good way to traverse and return all acceptable low correlation feature combinations very quickly. This will aid anyone looking to iterate through a lot of models as they test various stacking strategies.